[Shelter Animal Outcomes](https://www.kaggle.com/c/shelter-animal-outcomes)

In [1]:
reset -fs

In [2]:
import pandas as pd
import numpy as np
# import datetime

import matplotlib.pyplot as plt
import seaborn as sns

import plotly.plotly as py
import plotly.graph_objs as go
import cufflinks as cf

sns.set_style("dark")
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [3]:
animals_test = pd.read_csv('data/test.csv')
print('shape:',animals_test.shape)
animals_test.head(2)

shape: (11456, 8)


,ID,Name,DateTime,AnimalType,SexuponOutcome,AgeuponOutcome,Breed,Color
0,1,Summer,2015-10-12 12:15:00,Dog,Intact Female,10 months,Labrador Retriever Mix,Red/White
1,2,Cheyenne,2014-07-26 17:59:00,Dog,Spayed Female,2 years,German Shepherd/Siberian Husky,Black/Tan


In [4]:
animals = pd.read_csv('data/train.csv')
animals.head(2)

,AnimalID,Name,DateTime,OutcomeType,OutcomeSubtype,AnimalType,SexuponOutcome,AgeuponOutcome,Breed,Color
0,A671945,Hambone,2014-02-12 18:22:00,Return_to_owner,NaN,Dog,Neutered Male,1 year,Shetland Sheepdog Mix,Brown/White
1,A656520,Emily,2013-10-13 12:44:00,Euthanasia,Suffering,Cat,Spayed Female,1 year,Domestic Shorthair Mix,Cream Tabby


In [5]:
animals.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26729 entries, 0 to 26728
Data columns (total 10 columns):
AnimalID          26729 non-null object
Name              19038 non-null object
DateTime          26729 non-null object
OutcomeType       26729 non-null object
OutcomeSubtype    13117 non-null object
AnimalType        26729 non-null object
SexuponOutcome    26728 non-null object
AgeuponOutcome    26711 non-null object
Breed             26729 non-null object
Color             26729 non-null object
dtypes: object(10)
memory usage: 2.0+ MB


In [6]:
# rename columns
animals.columns = [x.lower() for x in animals.columns]
animals = animals.rename(index=str, 
               columns={
                   "animalid": "id",
                   "datetime": "date_outcome",
                   "outcometype": "outcome",
                   "outcomesubtype": "outcome_info",
                   "animaltype": "animal",
                   "sexuponoutcome": "sex_outcome",
                   "ageuponoutcome": "age"})
animals.columns

Index(['id', 'name', 'date_outcome', 'outcome', 'outcome_info', 'animal',
       'sex_outcome', 'age', 'breed', 'color'],
      dtype='object')

In [7]:
animals.name.value_counts(dropna=False).head()

NaN        7691
Max         136
Bella       135
Charlie     107
Daisy       106
Name: name, dtype: int64

In [8]:
# replace NaN with "unknown"
animals.name.replace(np.nan, 'unknown', inplace=True)
animals.name.value_counts(dropna=False).head()

unknown    7691
Max         136
Bella       135
Charlie     107
Daisy       106
Name: name, dtype: int64

### Sex and spayed/neutered
make new columns for sex, spayed/neutered... drop unknowns

In [9]:
animals.sex_outcome.value_counts(dropna=False)

Neutered Male    9779
Spayed Female    8820
Intact Male      3525
Intact Female    3511
Unknown          1093
NaN                 1
Name: sex_outcome, dtype: int64

In [10]:
# make "Unknown" NaN for dropping
animals.sex_outcome.replace('Unknown', np.nan, inplace=True)

In [11]:
animals.shape

(26729, 10)

In [12]:
animals.dropna(subset=['sex_outcome'],inplace=True)

In [13]:
animals.shape

(25635, 10)

In [14]:
animals['sex'] = animals.sex_outcome.apply(lambda row: 1 if 'Female' in row else 0)
animals['spay_neutered'] = animals.sex_outcome.apply(lambda row: 0 if 'Intact' in row else 1)

In [15]:
animals.sex.value_counts(dropna=False)

0    13304
1    12331
Name: sex, dtype: int64

In [16]:
animals.spay_neutered.value_counts(dropna=False)

1    18599
0     7036
Name: spay_neutered, dtype: int64

### Date of outcome
convert string date to datetime and create columns for day, month, year

In [18]:
animals['datetime'] = pd.to_datetime(animals['date_outcome'], infer_datetime_format=True)

In [22]:
# Monday is 0, Sunday is 6
animals['day'] = animals['datetime'].apply(lambda row: row.weekday())
animals['month'] = animals['datetime'].apply(lambda row: row.month)
animals['year'] = animals['datetime'].apply(lambda row: row.year)
# morning, afternoon, evening
animals['hour'] = animals['datetime'].apply(lambda row: row.hour)
# to investigate when hour is zero some more
animals['minutes'] = animals['datetime'].apply(lambda row: row.minute)

In [28]:
animals.datetime.value_counts().head(10)

2015-08-11 00:00:00    19
2015-11-17 00:00:00    17
2015-07-02 00:00:00    13
2015-04-02 00:00:00    11
2014-08-31 09:00:00    10
2015-11-23 00:00:00     9
2016-02-15 00:00:00     9
2015-09-11 00:00:00     9
2015-09-01 00:00:00     8
2015-11-08 00:00:00     8
Name: datetime, dtype: int64

In [35]:
animals.hour.value_counts().sort_index()

0      378
5        2
6       10
7      110
8      302
9     1278
10     405
11    2042
12    2513
13    2468
14    2800
15    2682
16    2690
17    4162
18    3684
19    1083
20      77
21      18
22       5
23      20
Name: hour, dtype: int64

In [38]:
animals[(animals['hour']==0) & (animals['minutes']==0)]

,id,name,date_outcome,outcome,outcome_info,animal,sex,age,breed,color,datetime,day,month,year,hour,minutes
38,A702655,Mica,2015-08-11 00:00:00,Transfer,Partner,Cat,Neutered Male,4 months,Domestic Shorthair Mix,Black/White,2015-08-11,1,8,2015,0,0
117,A713392,Basil,2015-11-05 00:00:00,Transfer,Partner,Dog,Intact Male,3 months,Chihuahua Shorthair Mix,Black/White,2015-11-05,3,11,2015,0,0
175,A710071,Herb,2015-09-01 00:00:00,Transfer,Partner,Cat,Neutered Male,2 months,Domestic Shorthair Mix,White/Black,2015-09-01,1,9,2015,0,0
181,A716575,Scarlet,2015-12-22 00:00:00,Transfer,Partner,Dog,Spayed Female,2 years,Australian Cattle Dog Mix,Tan/White,2015-12-22,1,12,2015,0,0
195,A683666,unknown,2015-09-01 00:00:00,Transfer,Partner,Cat,Neutered Male,1 year,Domestic Shorthair Mix,White/Cream Tabby,2015-09-01,1,9,2015,0,0
248,A709067,Earlene,2015-08-05 00:00:00,Return_to_owner,NaN,Dog,Intact Female,5 months,Boxer Mix,Black/White,2015-08-05,2,8,2015,0,0
251,A720190,Little Missy,2016-02-07 00:00:00,Transfer,Partner,Dog,Spayed Female,7 years,Airedale Terrier Mix,Tricolor,2016-02-07,6,2,2016,0,0
289,A714820,Lizzie,2015-11-05 00:00:00,Transfer,Partner,Dog,Spayed Female,6 months,Plott Hound Mix,Brown Brindle,2015-11-05,3,11,2015,0,0
306,A716488,unknown,2015-11-23 00:00:00,Transfer,Partner,Dog,Neutered Male,2 years,Pit Bull Mix,Red,2015-11-23,0,11,2015,0,0
409,A716400,Calliope,2015-11-24 00:00:00,Transfer,Partner,Dog,Intact Female,1 year,Yorkshire Terrier Mix,Black/Tan,2015-11-24,1,11,2015,0,0


In [39]:
animals[(animals['hour']==0) & (animals['minutes']==0)].shape

(375, 16)

In [40]:
animals.animal.value_counts(dropna=False)

Dog    15595
Cat    11134
Name: animal, dtype: int64

In [41]:
animals.outcome.value_counts(dropna=False)

Adoption           10769
Transfer            9422
Return_to_owner     4786
Euthanasia          1555
Died                 197
Name: outcome, dtype: int64

In [42]:
animals.outcome_info.value_counts(dropna=False)

NaN                    13612
Partner                 7816
Foster                  1800
SCRP                    1599
Suffering               1002
Aggressive               320
Offsite                  165
In Kennel                114
Behavior                  86
Rabies Risk               74
Medical                   66
In Foster                 52
Enroute                    8
Court/Investigation        6
At Vet                     4
In Surgery                 3
Barn                       2
Name: outcome_info, dtype: int64

In [43]:
animals[['id']].groupby([animals['outcome'], animals['outcome_info']]).count()

id
outcome    outcome_info             
Adoption   Barn                    1
           Foster               1800
           Offsite               165
Died       At Vet                  4
           Enroute                 8
           In Foster              52
           In Kennel             114
           In Surgery              3
Euthanasia Aggressive            320
           Behavior               86
           Court/Investigation     6
           Medical                66
           Rabies Risk            74
           Suffering            1002
Transfer   Barn                    1
           Partner              7816
           SCRP                 1599

In [44]:
animals[['id']].groupby([animals['outcome'], animals['outcome_info']]).count().unstack()

id                                                   \
outcome_info Aggressive At Vet Barn Behavior Court/Investigation Enroute   
outcome                                                                    
Adoption            NaN    NaN  1.0      NaN                 NaN     NaN   
Died                NaN    4.0  NaN      NaN                 NaN     8.0   
Euthanasia        320.0    NaN  NaN     86.0                 6.0     NaN   
Transfer            NaN    NaN  1.0      NaN                 NaN     NaN   

                                                                             \
outcome_info  Foster In Foster In Kennel In Surgery Medical Offsite Partner   
outcome                                                                       
Adoption      1800.0       NaN       NaN        NaN     NaN   165.0     NaN   
Died             NaN      52.0     114.0        3.0     NaN     NaN     NaN   
Euthanasia       NaN       NaN       NaN        NaN    66.0     NaN     NaN   
Transfer         NaN       NaN       NaN        NaN     NaN     NaN  7816.0   

                                            
outcome_info Rabies Risk    SCRP Suffering  
outcome                                     
Adoption             NaN     NaN       NaN  
Died                 NaN     NaN       NaN  
Euthanasia          74.0     NaN    1002.0  
Transfer             NaN  1599.0       NaN

In [45]:
animals_ouctome = animals[['outcome', 'outcome_info']]

In [46]:
animals_ouctome

,outcome,outcome_info
0,Return_to_owner,NaN
1,Euthanasia,Suffering
2,Adoption,Foster
3,Transfer,Partner
4,Transfer,Partner
5,Transfer,Partner
6,Transfer,Partner
7,Transfer,Partner
8,Adoption,NaN
9,Adoption,Offsite


In [47]:
animals[['id']].groupby([animals['outcome'], animals['outcome_info'].isnull()]).count()

id
outcome         outcome_info      
Adoption        False         1966
                True          8803
Died            False          181
                True            16
Euthanasia      False         1554
                True             1
Return_to_owner True          4786
Transfer        False         9416
                True             6

In [48]:
animals[['outcome', 'outcome_info']][animals['outcome_info'].isnull()]

,outcome,outcome_info
0,Return_to_owner,NaN
8,Adoption,NaN
11,Adoption,NaN
13,Return_to_owner,NaN
14,Return_to_owner,NaN
19,Return_to_owner,NaN
22,Adoption,NaN
23,Adoption,NaN
24,Return_to_owner,NaN
26,Adoption,NaN


In [40]:
def outcome_info_cleanup(orginal='original', new='new'):
    animals.outcome_info = animals.outcome_info.str.replace(orginal, new)
    return len(animals[animals['outcome_info'] == new])

In [42]:
outcome_info_cleanup(orginal='Aggressive', new='Behavior')

406

In [43]:
outcome_info_cleanup(orginal='In Foster', new='Foster')

1852

In [44]:
outcome_info_cleanup(orginal='At Vet', new='Medical')

70

In [45]:
outcome_info_cleanup(orginal='In Surgery', new='Medical')

73

In [47]:
animals.outcome_info.value_counts(dropna=False)

NaN                    13612
Partner                 7816
Foster                  1852
SCRP                    1599
Suffering               1002
Behavior                 406
Offsite                  165
In Kennel                114
Rabies Risk               74
Medical                   73
Enroute                    8
Court/Investigation        6
Barn                       2
Name: outcome_info, dtype: int64

In [45]:
animals.columns

Index(['id', 'name', 'date', 'outcome', 'outcome_info', 'animal', 'sex', 'age',
       'breed', 'color', 'datetime', 'day', 'month', 'year', 'hour',
       'minutes'],
      dtype='object')

In [46]:
animals.sex.value_counts(dropna=True)

Neutered Male    9779
Spayed Female    8820
Intact Male      3525
Intact Female    3511
Unknown          1093
Name: sex, dtype: int64

In [47]:
len(animals.breed.value_counts(dropna=True))

1380

In [48]:
len(animals.color.value_counts(dropna=True))

366

In [49]:
animals.name.value_counts().head(10)

unknown     7691
Max          136
Bella        135
Charlie      107
Daisy        106
Lucy          94
Buddy         87
Princess      86
Rocky         85
Luna          68
Name: name, dtype: int64

In [50]:
animals.shape

(26729, 16)

0.8.2


In [53]:
animals.columns

Index(['id', 'name', 'date', 'outcome', 'outcome_info', 'animal', 'sex', 'age',
       'breed', 'color', 'datetime', 'day', 'month', 'year', 'hour',
       'minutes'],
      dtype='object')

In [54]:
animals.sex.value_counts()

Neutered Male    9779
Spayed Female    8820
Intact Male      3525
Intact Female    3511
Unknown          1093
Name: sex, dtype: int64

In [81]:
animals.age.value_counts()

1 year       3969
2 years      3742
2 months     3397
3 years      1823
1 month      1281
3 months     1277
4 years      1071
5 years       992
4 months      888
6 years       670
3 weeks       659
5 months      652
6 months      588
8 years       536
7 years       531
2 weeks       529
10 months     457
10 years      446
8 months      402
4 weeks       334
9 years       288
7 months      288
12 years      234
9 months      224
1 weeks       171
11 months     166
1 week        146
13 years      143
11 years      126
3 days        109
2 days         99
14 years       97
15 years       85
1 day          66
4 days         50
6 days         50
16 years       36
5 days         24
0 years        22
17 years       17
5 weeks        11
18 years       10
19 years        3
20 years        2
Name: age, dtype: int64

## TO DO: explore other plot types & colors...
- ie boxplots?
- regplot, lmplot (logistic regression plot from seaborn?)

In [56]:
animals['sex'].value_counts().iplot(kind='bar', yTitle='Sex', title='Animal sex',)

In [55]:
animals['name'].value_counts()[:20].iplot(kind='bar', yTitle='Name Frequency', title='Animal names',)

In [57]:
animals['outcome'].value_counts().iplot(kind='bar', yTitle='Outcomes', title='Animal outcomes',)

In [59]:
# subset/group by dog vs cat
animals['breed'].value_counts().iplot(kind='bar', yTitle='Breeds', title='Animal Breeds')

In [60]:
# subset/group by dog vs cat
animals['color'].value_counts().iplot(kind='bar', yTitle='Color of coat', title='Animal coat colors')